HW 4 (Used L4 GPU)

# Task 1

In [1]:
!pip install transformers torch -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import math

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


Models: Using greedy search, beam search, top-k sampling, and top-p sampling

1. greedy decoding if num_beams=1 and do_sample=False
2. beam-search decoding if num_beams>1 and do_sample=False


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((3072,), eps=1e-05)
    (rotary_emb

## Prompts

In [4]:
prompts = [
    "Yesterday, I was looking to",
    "In the final race of Formula 1, it all",
    "The comedian made a joke about",
    "The future of crypto is",
    "While we were chatting at the cafe, my friends"
]

In [5]:
def compute_perplexity(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

In [6]:
# Set pad_token_id in the model config
model.config.pad_token_id = tokenizer.eos_token_id
results = []

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Greedy Search
    greedy_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        max_length=25
    )
    greedy_text = tokenizer.batch_decode(greedy_output, skip_special_tokens=True)
    greedy_ppl = compute_perplexity(model, tokenizer, greedy_text)
    print("Greedy Search: " + greedy_text[0])

    # Beam Search
    beam_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.eos_token_id,
        max_length=25,
        num_beams=3,
        early_stopping=True,
        do_sample=False
    )
    beam_text = tokenizer.decode(beam_output[0], skip_special_tokens=True)
    beam_ppl = compute_perplexity(model, tokenizer, beam_text)
    print("Beam Search: " + beam_text)

    # Top K Sampling
    top_k_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.eos_token_id,
        max_length=25,
        do_sample=True,
        top_k=10,
        top_p=1.0
    )
    top_k_text = tokenizer.decode(top_k_output[0], skip_special_tokens=True)
    top_k_ppl = compute_perplexity(model, tokenizer, top_k_text)
    print("Tok k Sample: " + top_k_text)

    # Top P Sampling
    top_p_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        pad_token_id=tokenizer.eos_token_id,
        max_length=25,
        do_sample=True,
        top_k=0,
        top_p=0.7
    )
    top_p_text = tokenizer.decode(top_p_output[0], skip_special_tokens=True)
    top_p_ppl = compute_perplexity(model, tokenizer, top_p_text)
    print("Top p Sample: " + top_p_text)

    results.append({
        "Prompt": prompt,
        "Greedy Text": greedy_text, "Greedy PPL": greedy_ppl,
        "Beam Text": beam_text, "Beam PPL": beam_ppl,
        "Top-k Text": top_k_text, "Top-k PPL": top_k_ppl,
        "Top-p Text": top_p_text, "Top-p PPL": top_p_ppl,
    })

    print("-" * 80)

/home/vamsik1211/Data/miniconda3/envs/nlp-class/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vamsik1211/Data/miniconda3/envs/nlp-class/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Greedy Search: Yesterday, I was looking to get some exercise and decided to go for a run. I had been feeling a bit sluggish
Beam Search: Yesterday, I was looking to buy a new pair of shoes, and I stumbled upon a great deal on a pair of
Tok k Sample: Yesterday, I was looking to get some work done on my computer, and as I was getting settled in, I noticed
Top p Sample: Yesterday, I was looking to try out a new recipe for dinner, and I stumbled upon a recipe for a delicious-s
--------------------------------------------------------------------------------
Greedy Search: In the final race of Formula 1, it all comes down to the last lap. The tension is palpable as
Beam Search: In the final race of Formula 1, it all comes down to the last lap. The tension is palpable as
Tok k Sample: In the final race of Formula 1, it all comes down to the last two drivers, Lewis Hamilton and Charles Le
Top p Sample: In the final race of Formula 1, it all comes down to the last few laps. The tension is palpa

In [7]:
import pandas as pd

df = pd.DataFrame(results)

from IPython.display import display
display(df)

,Prompt,Greedy Text,Greedy PPL,Beam Text,Beam PPL,Top-k Text,Top-k PPL,Top-p Text,Top-p PPL
0,"Yesterday, I was looking to","[Yesterday, I was looking to get some exercise...",5.525412,"Yesterday, I was looking to buy a new pair of ...",5.299848,"Yesterday, I was looking to get some work done...",7.044036,"Yesterday, I was looking to try out a new reci...",5.891177
1,"In the final race of Formula 1, it all","[In the final race of Formula 1, it all comes ...",7.595809,"In the final race of Formula 1, it all comes d...",7.595809,"In the final race of Formula 1, it all comes d...",8.951245,"In the final race of Formula 1, it all comes d...",7.567790
2,The comedian made a joke about,[The comedian made a joke about the new iPhone...,7.089778,The comedian made a joke about his wife's cook...,5.307983,The comedian made a joke about his ex-girlfrie...,10.015948,"The comedian made a joke about the ""Glorious"" ...",10.762782
3,The future of crypto is,"[The future of crypto is uncertain, but one th...",3.689758,"The future of crypto is uncertain, but one thi...",3.434429,"The future of crypto is uncertain, but one thi...",3.946754,"The future of crypto is uncertain, but one thi...",3.689758
4,"While we were chatting at the cafe, my friends","[While we were chatting at the cafe, my friend...",9.769113,"While we were chatting at the cafe, my friends...",8.871318,"While we were chatting at the cafe, my friends...",9.706848,"While we were chatting at the cafe, my friends...",9.003258


# Task 2

In [26]:
!pip install transformers datasets sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.9 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset

xsum_dataset = load_dataset("xsum")
test_data = xsum_dataset["test"]
print(test_data)

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 11334
})


In [9]:
#model_name = "gniemiec/t5-small-finetuned-xsum"
model_name = "google/pegasus-xsum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model.to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_nor

In [10]:
model.config.pad_token_id = tokenizer.eos_token_id

num_samples = 50
results = []

for i in range(num_samples):
    document = test_data[i]["document"]
    reference_summary = test_data[i]["summary"]

    inputs = tokenizer(document, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Greedy Search
    greedy_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=25,
        do_sample=False,
    )
    greedy_text = tokenizer.decode(greedy_output[0], skip_special_tokens=True)

    # Beam Search
    beam_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=25,
        num_beams=3,
        early_stopping=True,
        do_sample=False
    )
    beam_text = tokenizer.decode(beam_output[0], skip_special_tokens=True)

    # Top-k Sampling
    top_k_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=25,
        do_sample=True,
        top_k=10,
        top_p=1.0
    )
    top_k_text = tokenizer.decode(top_k_output[0], skip_special_tokens=True)

    # Top-p Sampling
    top_p_output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=25,
        do_sample=True,
        top_k=0,
        top_p=0.7
    )
    top_p_text = tokenizer.decode(top_p_output[0], skip_special_tokens=True)

    print(f"Document #{i+1}: {document[:100]}...")
    print("Reference Summary:", reference_summary[:100], "...")
    print("Greedy:", greedy_text)
    print("Beam:", beam_text)
    print("Top-k:", top_k_text)
    print("Top-p:", top_p_text)
    print("-" * 80)

    results.append({
        "Document": document,
        "Reference Summary": reference_summary,
        "Greedy Output": greedy_text,
        "Beam Output": beam_text,
        "Top-k Output": top_k_text,
        "Top-p Output": top_p_text,
    })

/home/vamsik1211/Data/miniconda3/envs/nlp-class/lib/python3.12/site-packages/transformers/generation/utils.py:1532: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Document #1: Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up...
Reference Summary: There is a "chronic" need for more housing for prison leavers in Wales, according to a charity. ...
Greedy: There is a "desperate need" for more affordable housing for men leaving prison, a charity has said.
Beam: More than 1,000 homeless people were referred to a charity in Wales last year.
Top-k: More than 1,000 homeless people were referred to a charity in Wales last year.
Top-p: There is a "desperate need" for more housing for prison leavers in Wales, a charity has said
--------------------------------------------------------------------------------
Document #2: Officers searched properties in the Waterfront Park and Colonsay View areas of the city on Wednesday...
Reference Summary: A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh. ...
Greedy: A man has been charged after firearms and money were sei

In [11]:
import pandas as pd

df = pd.DataFrame(results)
df

,Document,Reference Summary,Greedy Output,Beam Output,Top-k Output,Top-p Output
0,"Prison Link Cymru had 1,099 referrals in 2015-...","There is a ""chronic"" need for more housing for...","There is a ""desperate need"" for more affordabl...","More than 1,000 homeless people were referred ...","More than 1,000 homeless people were referred ...","There is a ""desperate need"" for more housing f..."
1,Officers searched properties in the Waterfront...,"A man has appeared in court after firearms, am...",A man has been charged after firearms and mone...,A man has been charged after firearms and mone...,A man has been charged after firearms and mone...,A man has been charged after firearms and mone...
2,"Jordan Hill, Brittany Covington and Tesfaye Co...",Four people accused of kidnapping and torturin...,Four black people accused of torturing a menta...,Four black people accused of torturing a menta...,Four black people accused of torturing a menta...,Four black people accused of torturing a menta...
3,The 48-year-old former Arsenal goalkeeper play...,West Brom have appointed Nicky Hammond as tech...,West Bromwich Albion have appointed Reading di...,West Bromwich Albion have appointed Reading di...,West Brom have appointed Reading director of f...,West Bromwich Albion have appointed Reading di...
4,Restoring the function of the organ - which he...,The pancreas can be triggered to regenerate it...,"The ""fasting-mimicking"" diet can regenerate pa...","The ""fasting-mimicking"" diet can regenerate th...","The ""fasting-mimicking"" diet can regenerate pa...","The ""fasting-mimicking"" diet can regenerate pa..."
5,But there certainly should be.\nThese are two ...,Since their impending merger was announced in ...,There is not much buzz about the planned merge...,There is not a lot of buzz about the planned m...,There is not much buzz about the planned merge...,There is not a lot to say about the planned me...
6,Media playback is not supported on this device...,"A ""medal at any cost"" approach created a ""cult...",naghel Houvenaghel has become the latest high-...,naghel Houvenaghel has become the latest high-...,naghel Houvenaghel has become the latest high-...,naghel Houvenaghel has become the latest high-...
7,It's no joke. But Kareem Badr says people did ...,Have you heard the one about the computer prog...,"If you go to a comedy club in Texas, you're li...",If you've ever been to a comedy club in the US...,"When you buy a comedy club in Texas, you don't...","If you go to a comedy club in the US, you're p..."
8,Relieved that the giant telecoms company would...,The reaction from BT's investors told us much ...,Ofcom's decision not to press for a break-up o...,When the regulator Ofcom said it would not be ...,Ofcom's decision not to press for a break-up o...,Ofcom's decision not to press for a break-up o...
9,"""I'm really looking forward to it - the home o...",Manager Brendan Rodgers is sure Celtic can exp...,Celtic manager Brendan Rodgers is relishing th...,Celtic manager Brendan Rodgers is relishing th...,Celtic manager Brendan Rodgers is relishing th...,Celtic manager Brendan Rodgers is relishing th...


In [14]:
df.to_csv("results.csv", index=False)

# Markdown

In [ ]:
import evaluate
import pandas as pd

# Load evaluation metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

# Initialize results list
evaluation_results = []

# Iterate over the generated results
for result in results:
    predictions = [
        result["Greedy Output"],
        result["Beam Output"],
        result["Top-k Output"],
        result["Top-p Output"]
    ]
    references = [result["Reference Summary"]] * len(predictions)

    # Compute BLEU
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    
    # Compute ROUGE
    rouge_score = rouge.compute(predictions=predictions, references=references)
    
    # Compute BERTScore
    bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")

    # Append scores to the result
    evaluation_results.append({
        "Document": result["Document"],
        "Reference Summary": result["Reference Summary"],
        "Greedy Output": result["Greedy Output"],
        "Beam Output": result["Beam Output"],
        "Top-k Output": result["Top-k Output"],
        "Top-p Output": result["Top-p Output"],
        # "BLEU": bleu_score["bleu"],
        "ROUGE-L": rouge_score["rougeL"],
        "BERTScore (F1)": sum(bertscore_result["f1"]) / len(bertscore_result["f1"])
    })

# Convert results to a DataFrame
evaluation_df = pd.DataFrame(evaluation_results)

# Save to CSV
evaluation_df.to_csv("evaluation_results.csv", index=False)

# Display averaged scores
# average_bleu = evaluation_df["BLEU"].mean()
average_rouge_l = evaluation_df["ROUGE-L"].mean()
average_bertscore_f1 = evaluation_df["BERTScore (F1)"].mean()

# print(f"Average BLEU: {average_bleu}")
print(f"Average ROUGE-L: {average_rouge_l}") # Using Rouge-L because xsum dataset uses it in the published research paper.
print(f"Average BERTScore (F1): {average_bertscore_f1}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Average ROUGE-L: 0.40450627159310976
Average BERTScore (F1): 0.9155059200525284
